# Settle Up Problem (Dlužníček)

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [5]:
!pip install gurobipy
P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values())/len(c)  # the settlement value

Defaulting to user installation because normal site-packages is not writeable


Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x, y, z)$ representing the transactions: person $x$ should pay person $y$ amount of $z$ euros. The number of transactions (i.e., the length of the list) should be minimized.

For the given instance, the **optimal solution** consists of 3 transactions, namely

```
A -> B: 250.0
C -> D: 50.0
C -> B: 90.0
```

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [ ]:
import gurobipy as g  # import Gurobi module

# model --------------------------------------------------
m = g.Model()

# - ADD VARIABLES
t_a = m.addVars(len(c), len(c), vtype=g.GRB.INTEGER, lb=0)  # velikost transackce
t_s = m.addVars(len(c), len(c), vtype=g.GRB.BINARY, lb=0)  # probla transakce

M = c.sum()

# - ADD CONSTRAINTS
for person_id in range(len(c)):
    m.addConstr(t_a.sum(person_id, "*") -
                t_a.sum("*", person_id) + c[person_id] == sv)

for i in range(len(c)):
   for j in range(len(c)):
      m.addConstr(t_a[i][j] <= M * t_s[i][j])


# - SET OBJECTIVE
# chceme minimalizovat pocet transakci
m.setObjective(t_s.sum())

# call the solver -------------------------------------------
m.optimize()

# print the solution -----------------------------------------
print('\nSOLUTION:')
print([t_a[i,j].x for i in range(len(c)) for j in range(len(c))])

KeyError: 0